In [42]:
# !pip install -U sentence-transformers
!pip install gradio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [45]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from sentence_transformers import evaluation
from sentence_transformers.cross_encoder import CrossEncoder
import ast
from sentence_transformers import models, losses, InputExample
from torch.utils.data import DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import sent_tokenize, word_tokenize
from transformers import AutoTokenizer, AutoModelWithLMHead
from nltk.stem import PorterStemmer
import torch
import torch.nn as nn
import nltk
import re
import gradio as gr
#from GPUtil import showUtilization as gpu_usage
#from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()
nltk.download('words')
words = set(nltk.corpus.words.words())

def stemSentence_porter(sentence):
    porter = PorterStemmer()
    token_words=word_tokenize(sentence)
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

def convert(text):
    L = []
    for i in ast.literal_eval(text):
        L.append(i['name']) 
    return L 


def fetch_director(text):
    L = []
    for i in ast.literal_eval(text):
        if i['job'] == 'Director':
            L.append(i['name'])
    return L 


def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

def clean_text(text):
    # remove backslash-apostrophe 
    text = re.sub("\'", "", text) 
    # remove everything except alphabets 
    text = re.sub("[^a-zA-Z0-9,.’]"," ",text) 
    # remove whitespaces 
    text = ' '.join(text.split()) 
    # convert text to lowercase 
    
    return text

def get_cosine_sim(model, df):
    scores = []
    for row in zip(df['tags'], df['reviews']):
        scores.append(util.cos_sim(model.encode(row[0]), model.encode(row[1])))
    return scores

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps" if torch.backends.mps.is_available() else device)
device

[nltk_data] Downloading package words to
[nltk_data]     /Users/zijianchen/nltk_data...
[nltk_data]   Package words is already up-to-date!


device(type='mps')

In [ ]:
movies = pd.read_csv('../tmdb_5000_data/tmdb_5000_movies.csv')
credits = pd.read_csv('../tmdb_5000_data/tmdb_5000_credits.csv')
movies = movies.merge(credits,on='title')

reviews = pd.read_csv('../crawled_data/2022-11-19_movie_info_with_reviews.csv')
reviews = reviews[['id','reviews']]
reviews['reviews'] = reviews['reviews'].apply(lambda x: list(map(clean_text, x.split("\',"))))
movies = movies.merge(reviews,on='id', how='left')

movies.dropna(inplace=True)
movies['release_year'] = movies.release_date.apply(lambda x: x.split("-")[0]).astype(int)
movies = movies[movies['release_year']>1970]

movies = movies[['movie_id','title','release_year','overview','genres','keywords','cast','crew','reviews']]
movies.dropna(inplace=True)

movies['genres'] = movies['genres'].apply(convert)

movies['keywords'] = movies['keywords'].apply(convert)

movies['cast'] = movies['cast'].apply(convert)

movies['cast'] = movies['cast'].apply(lambda x:x[0:3])

movies['crew'] = movies['crew'].apply(fetch_director)


movies['crew'] = movies['crew'].apply(lambda x: [" ".join(\
        w for w in nltk.wordpunct_tokenize(i) \
         if w.lower() in words or not w.isalpha()) for i in x])

In [ ]:
storyline = pd.read_csv('../tmdb_5000_data/2022-12-04movie_with_imdb_storyline.csv', index_col=0) 
# storyline['storyline'] = storyline['storyline'].apply(lambda x: list(map(clean_text, x.split("\',"))))

movies = movies.merge(storyline[['movie_id','title','storyline']], how='left', left_on=['movie_id','title'], right_on = ['movie_id','title'])
movies['overview'] = movies['overview'].apply(lambda x:x.split())
movies['storyline'] = movies['storyline'].apply(lambda x:str(x).split())

movies['release_year'] = movies['release_year'].astype(str).apply(lambda x:x.split())
# movies['tags'] = movies['overview'] + movies['genres'] + movies['release_year'] + movies['keywords']
movies['tags'] = movies['overview'] + pd.Series(np.repeat(' The genre is ', len(movies), axis =0), index = movies.index).apply(lambda x:x.split()) \
+ movies['genres'] + pd.Series(np.repeat('.', len(movies), axis =0), index = movies.index).apply(lambda x:x.split())  \
+ pd.Series(np.repeat(' The movie was released in ', len(movies), axis =0), index = movies.index).apply(lambda x:x.split()) + pd.Series(np.repeat('.', len(movies), axis =0), index = movies.index).apply(lambda x:x.split()) \
+ movies['release_year'] + pd.Series(np.repeat(' The charactistics of the movie are ', len(movies), axis =0), index = movies.index).apply(lambda x:x.split()) \
+ movies['keywords'] + pd.Series(np.repeat('.', len(movies), axis =0), index = movies.index).apply(lambda x:x.split())  + movies['storyline']

new = movies.drop(columns=['overview','genres','keywords','cast','crew','release_year','storyline'])


In [ ]:
new['tags'] = new['tags'].apply(lambda x: " ".join(x))
new = new[(new['tags'].notnull())].reset_index(drop=True)
new = new[new["reviews"].str.len() != 0].reset_index(drop=True)

In [ ]:
new = new[new["reviews"].str.len() > 1]

In [ ]:
new = new.explode(['reviews'])
# new = new[(new['reviews']!='')].reset_index(drop=True)
# new.to_csv("../tmdb_5000_data/Cleaned_Filtered_Plots_Reviews")

In [ ]:
new

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-imdb-sentiment")

model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-imdb-sentiment")
model = model.to(device)
def get_sentiment(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt').to(device)

  output = model.generate(input_ids=input_ids,
               max_length=2)
  
  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  return label

In [ ]:
new['review_sentiment'] = new['reviews'].apply(lambda x: get_sentiment(x)[6:])

In [ ]:
new['sentiment_score'] = new['review_sentiment'].map({'positive': 1, 'negative': 0})

In [ ]:
new = new[(new['reviews'].str.len() > 1)].reset_index(drop=True)

In [ ]:
new

In [ ]:
#new = new.drop(['Unnamed: 0'], axis=1)

new['grouped_reviews'] = new.groupby(['movie_id','title','tags','review_sentiment','sentiment_score'])['reviews'].transform(lambda x: '.'.join(x)).drop_duplicates()

In [ ]:
new = new.dropna().drop(['reviews'], axis=1)
new

In [ ]:
new.to_csv("../tmdb_5000_data/tmdb_sentiment_grouped_reviews.csv",index=False)

In [19]:
new = pd.read_csv('../tmdb_5000_data/tmdb_sentiment_grouped_reviews.csv') 

In [20]:
new

,movie_id,title,tags,review_sentiment,sentiment_score,grouped_reviews
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",positive,1.0,Cameron’s epic can still thrill the audience w...
1,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",negative,0.0,Five hundred million dollars wasted ..The leve...
2,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,positive,1.0,It still might not be quite the conclusion we ...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,negative,0.0,Even with a twisting mystery and numerous new ...
4,49529,John Carter,"John Carter is a war-weary, former military ca...",positive,1.0,John Carter is a good summer movie in March bu...
...,...,...,...,...,...,...
1723,2292,Clerks,Convenience and video store clerks Dante and R...,negative,0.0,...the films inherent deficiencies are general...
1724,14337,Primer,Friends/fledgling entrepreneurs invent a devic...,positive,1.0,Time travel may provide the paradoxical mechan...
1725,14337,Primer,Friends/fledgling entrepreneurs invent a devic...,negative,0.0,The storytelling is so confusing and the multi...
1726,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,negative,0.0,A star is born in Daniel Henney in the predict...


In [21]:
# microsoft/mpnet-base
# sentence-transformers/stsb-bert-large
word_embedding_model = models.Transformer('sentence-transformers/all-mpnet-base-v2').to(device)

In [22]:
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), 
                                  pooling_mode_mean_tokens=True, 
                                  pooling_mode_cls_token=False, 
                                  pooling_mode_max_tokens=False
                              )

In [23]:
# update the model
sent_model = SentenceTransformer(modules=[word_embedding_model, pooling_model]).to(device)

In [24]:
train_examples = []

for index, row in enumerate(zip(new['tags'], new['grouped_reviews'], new['sentiment_score'])):
#     train_examples.append(InputExample(texts=[row[0], row[1]], label = [scores[index]]))
    train_examples.append(InputExample(texts=[row[0], row[1]], label = row[2]))    

In [25]:
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=3)
# ContrastiveLoss
# MarginMSELoss
# OnlineContrastiveLoss
# TripletLoss
train_loss = losses.OnlineContrastiveLoss(sent_model)

In [26]:
# dev_mse = evaluation.EmbeddingSimilarityEvaluator(new['tags'].values, new['reviews'].values,\
#                                                   scores=scores)
dev_mse = evaluation.BinaryClassificationEvaluator(new['tags'].values, \
                          new['grouped_reviews'].values, \
                          new['sentiment_score'].values)

In [27]:
# dev_mse = evaluation.MSEEvaluator(new['tags'].values, new['reviews'].values,teacher_model=sent_model)

In [28]:
sent_model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100, output_path='result')

Iteration:  16%|█████                          | 95/576 [05:33<26:17,  3.28s/it]


Iteration:  33%|██████████                    | 192/576 [10:42<21:18,  3.33s/it]


Iteration:  50%|███████████████               | 289/576 [16:09<16:33,  3.46s/it]


Iteration:  67%|████████████████████          | 386/576 [21:14<10:14,  3.24s/it]


Iteration:  84%|█████████████████████████▏    | 483/576 [26:22<04:48,  3.10s/it]


Epoch: 100%|████████████████████████████████████| 1/1 [31:03<00:00, 1863.12s/it]


In [29]:
torch.save(sent_model.state_dict(), 'best_sent_model.pt')

In [30]:
checkpoint = torch.load('best_sent_model.pt', device)
sent_model.load_state_dict(checkpoint)
sent_model.to(device)

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [31]:
retrieve_df = new[['movie_id','title','tags']].drop_duplicates().reset_index(drop=True)

descriptions = retrieve_df['tags']
des_embeddings = []
for i,des in enumerate(descriptions):
    des_embeddings.append(sent_model.encode(des))

In [32]:
def recommend(query, model):
    #Compute cosine-similarities with all embeddings 
    model.eval()
    query_embedd = model.encode(query)
    cosine_scores = util.pytorch_cos_sim(query_embedd, des_embeddings)
    top10_matches = torch.argsort(cosine_scores, dim=-1, descending=True).tolist()[0][0:10]
    return top10_matches

In [33]:
query_show_des = '3D Top notch education and entertainment for dinosaurs'
recommendded_results = recommend(query_show_des, sent_model)

for index in recommendded_results:
    print(retrieve_df.iloc[index,1])

Walking With Dinosaurs
Jurassic Park
The Good Dinosaur
Jurassic World
Yogi Bear
Thunder and the House of Magic
How to Train Your Dragon
The Secret of Kells
Finding Nemo
How to Train Your Dragon 2


/opt/anaconda3/envs/torch_m1_accelerate/lib/python3.10/site-packages/sentence_transformers/util.py:39: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1665990514504/work/torch/csrc/utils/tensor_new.cpp:233.)
  b = torch.tensor(b)


In [34]:
Walking With Dinosaurs
The Good Dinosaur
Jurassic Park
Jurassic World
Thunder and the House of Magic
Sanctum
Yogi Bear
The Secret of Kells
ParaNorman
Shrek Forever After

SyntaxError: invalid syntax (1182986673.py, line 1)

In [ ]:
query_show_des = 'alien attack earth'
recommendded_results = recommend(query_show_des, sent_model)

for index in recommendded_results:
    print(retrieve_df.iloc[index,1])

In [ ]:
The Darkest Hour
They Live
Taxi to the Dark Side
The Terminator
The Lazarus Effect
Riddick
Resident Evil
Meet the Spartans
30 Days of Night
Final Destination 5

In [ ]:
query_show_des = 'superhero fight trans genetic monster'
recommendded_results = recommend(query_show_des, sent_model)

for index in recommendded_results:
    print(retrieve_df.iloc[index,1])

In [ ]:
Clash of the Titans
Spider-Man
Hellboy
Blade II
Osmosis Jones
The Amazing Spider-Man
Prom Night
Watchmen
Boogeyman
American Hero

In [ ]:
query_show_des = 'glory space fighting in the solar system'
recommendded_results = recommend(query_show_des, sent_model)

for index in recommendded_results:
    print(retrieve_df.iloc[index,1])

In [ ]:
300
Red Cliff
Return of the Jedi
Letters from Iwo Jima
Beneath Hill 60
Riddick
Edge of Tomorrow
Apocalypse Now
Spaceballs
Space Cowboys

In [35]:
query_show_des = 'escape prison'
recommendded_results = recommend(query_show_des, sent_model)

for index in recommendded_results:
    print(retrieve_df.iloc[index,1])

Bronson
Taxi to the Dark Side
Escape Plan
Man on a Ledge
The Green Mile
American History X
Man on Wire
Sympathy for Lady Vengeance
Death Race
Side Effects


In [60]:
retrieve_df

,movie_id,title,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di..."
1,49026,The Dark Knight Rises,Following the death of District Attorney Harve...
2,49529,John Carter,"John Carter is a war-weary, former military ca..."
3,38757,Tangled,When the kingdom's most wanted-and most charmi...
4,767,Harry Potter and the Half-Blood Prince,"As Harry begins his sixth year at Hogwarts, he..."
...,...,...,...
905,158895,This Is Martin Bonner,"Two men, at opposite ends of the social spectr..."
906,42151,Down Terrace,After serving jail time for a mysterious crime...
907,2292,Clerks,Convenience and video store clerks Dante and R...
908,14337,Primer,Friends/fledgling entrepreneurs invent a devic...


In [71]:
def greet(sentence, justTitle):
    recommendded_results = recommend(sentence, sent_model)
    if justTitle:
        recommendded_results = [retrieve_df.iloc[index,1] for index in recommendded_results]
    else:
        recommendded_results = [retrieve_df.iloc[index,1]+': '+retrieve_df.iloc[index,2] for index in recommendded_results]
    return recommendded_results

with gr.Blocks() as demo:
    gr.Markdown("Typing below and then click **Run** to see the output.")
    with gr.Row():
        with gr.Column():
            inp = gr.Textbox(placeholder="Describe a piece of sence/plot you would like to watch",label = "Query")
            justTitle = gr.Checkbox(label="Return movie titles only?")
        with gr.Column():
            opt = gr.TextArea(placeholder="The movies you might like will be displayed here", label ="Recommendations:")
        btn = gr.Button("Run")
        btn.click(fn=greet, inputs=[inp,justTitle], outputs=[opt])


demo.launch(debug=True) 

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.
